# Dados de Entrada
* Selecione "Adicionar ao Drive"
* Links: 
  * https://tinyurl.com/bigdata-gut-pt
  * https://tinyurl.com/bigdata-amz





# Setup

## Instalação de pacotes

In [ ]:
!apt-get update > /dev/null
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!apt-get install netcat
!wget -q https://downloads.apache.org/spark/spark-3.2.4/spark-3.2.4-bin-hadoop3.2.tgz
!tar xf spark-3.2.4-bin-hadoop3.2.tgz
!pip install findspark pyspark 

## Acesso ao Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Preparação do ambiente

In [ ]:
%env PYTHONHASHSEED=1234
%env JAVA_HOME=/usr/lib/jvm/java-8-openjdk-amd64
%env SPARK_HOME=/content/spark-3.2.4-bin-hadoop3.2

In [ ]:
import findspark
findspark.init("/content/spark-3.2.4-bin-hadoop3.2")

In [ ]:
findspark.find()


# Streaming de arquivos

## Streaming Básico

In [ ]:
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.streaming import StreamingContext


In [ ]:
conf = SparkConf().setAppName('Big Data Streaming').setMaster('local[*]')
sc = SparkContext.getOrCreate(conf)

streaming = StreamingContext(sc, 1)

In [ ]:
!mkdir /content/streaming_test

In [ ]:
fstream = streaming.textFileStream('/content/streaming_test')



In [ ]:
import re

def words_in_line(line) :
  line = re.sub('[^a-zà-ù ]', ' ', line.lower())
  nwords = len(line.split())
  return (None, nwords)


In [ ]:
wc = fstream.map(words_in_line).reduceByKey(lambda acc, v: acc + v)


In [ ]:
wc.pprint()

In [ ]:
!rm -rf /content/streaming_test/*

Após iniciar o streaming, você deve copiar alguns arquivos para o diretório streaming_test

In [ ]:
streaming.start()             
query = streaming.awaitTermination(120)  

In [ ]:
sc.stop()
streaming.stop()

## Processamento para cada RDD no Stream

In [ ]:
conf = SparkConf().setAppName('Big Data Streaming').setMaster('local[*]')
sc = SparkContext.getOrCreate(conf)

streaming = StreamingContext(sc, 5)

In [ ]:
fstream = streaming.textFileStream('/content/streaming_test')


In [ ]:
!rm /content/streaming_test/*

In [ ]:
import re

def split_words(line) :
  line = re.sub('[^a-zà-ù ]', ' ', line.lower())
  words = line.split()
  for w in words :
    yield (w, 1)


In [ ]:
wc = fstream.flatMap(split_words) \
            .reduceByKey(lambda acc, v: acc + v)

In [ ]:
wc.pprint()

In [ ]:
all_res = []

In [ ]:
def process_rdd(time, rdd):
  global all_res
  if not rdd.isEmpty():
    rdd_sorted = rdd.sortBy(lambda pair: pair[1], ascending=False)
    all_res.append(rdd_sorted.take(20))


In [ ]:
wc.foreachRDD(process_rdd)

In [ ]:
!rm -rf /content/streaming_test/*

In [ ]:
streaming.start()             
query = streaming.awaitTermination(120)  

In [ ]:
all_res

In [ ]:
all_res_merged = sum(all_res, [])

In [ ]:
all_res_merged

In [ ]:
merged_rdd = sc.parallelize(all_res_merged)

In [ ]:
merged_rdd

In [ ]:
total_rdd = merged_rdd.reduceByKey(lambda acc, v : v+acc)

In [ ]:
total_rdd.take(10)

In [ ]:
sc.stop()
streaming.stop()

# Streaming via Rede

## Revisão Sobre sockets Client-Server



In [ ]:
import socket


In [ ]:
def server() :
  s = socket.socket()
  s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
  print("Socket Criado no Servidor")
  s.bind(('localhost',9998))
  s.listen(3)
  print("Esperando Conexões")

  while True:
    c, addr = s.accept()
    name = c.recv(1024).decode()
    print("Cliente ",addr, " falou: ", name)
    c.send(bytes('Eu sou o Servidor','utf-8'))
    c.close()

In [ ]:
def client() :
  c = socket.socket()
  c.connect(('localhost', 9998))
  name = "Eu sou o Cliente"
  c.send(bytes(name,'utf-8'))
  print('Servidor falou: '+c.recv(1024).decode())

In [ ]:
from multiprocessing import Process

sp = Process(target=server)
sp.start()



In [ ]:
cp = Process(target=client)
cp.start()


In [ ]:
cp = Process(target=client)
cp.start()

In [ ]:
sp.terminate()

In [ ]:
cp.join()
sp.join()


## Envio de um stream de dados


In [ ]:
import socket
import time

In [ ]:
def server_streaming() :
  s = socket.socket()
  s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
  print("Socket Criado no Servidor")
  s.bind(('localhost',9998))
  s.listen(3)
  print("Esperando Conexões")

  filename = '/content/drive/My Drive/amz/small.csv'
  with open(filename) as file:
    content = file.readlines()

  NLINES = 600

  while True:
    c, addr = s.accept()
    print("Conectado com ",addr)
    for i in range(NLINES) :
      c.send(bytes(content[i],'utf-8'))
      time.sleep(0.1)
    c.close()

In [ ]:
from multiprocessing import Process

sp = Process(target=server_streaming)
sp.start()



In [ ]:
!nc localhost 9998 

In [ ]:
sp.terminate()
sp.join()


# Spark Streaming via Rede

In [ ]:
from multiprocessing import Process

sp = Process(target=server_streaming)
sp.start()

In [ ]:
conf = SparkConf().setAppName('Big Data Streaming').setMaster('local[*]')
sc = SparkContext.getOrCreate(conf)

streaming = StreamingContext(sc, 10)

In [ ]:
netstream = streaming.socketTextStream("localhost", 9998)

In [ ]:
prod_rev = netstream.map(lambda line: (line.split(','))) \
              .map(lambda line: (line[0], (float(line[2]), 1))) \
              .reduceByKey(lambda acc, v: (acc[0] + v[0], acc[1] + v[1])) \
              .mapValues(lambda v: v[0]/v[1])


In [ ]:
prod_rev.pprint()

In [ ]:
streaming.start()             
query = streaming.awaitTermination(60)  

In [ ]:
sc.stop()
streaming.stop()

In [ ]:
sp.terminate()
sp.join()
